## Import dependencies

In [1]:
import pyspark
from pyspark.sql import HiveContext

## Read File

In [2]:
sc
sqlContext = HiveContext(sc)
bus_file='test.jsons'
bus = sqlContext.read.json(bus_file)
bus.registerTempTable("bus")

In [3]:
bus.printSchema()

root
 |-- Siri: struct (nullable = true)
 |    |-- ServiceDelivery: struct (nullable = true)
 |    |    |-- ResponseTimestamp: string (nullable = true)
 |    |    |-- SituationExchangeDelivery: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- Situations: struct (nullable = true)
 |    |    |    |    |    |-- PtSituationElement: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- Affects: struct (nullable = true)
 |    |    |    |    |    |    |    |    |-- VehicleJourneys: struct (nullable = true)
 |    |    |    |    |    |    |    |    |    |-- AffectedVehicleJourney: array (nullable = true)
 |    |    |    |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |    |    |    |-- DirectionRef: string (nullable = true)
 |    |    |    |    |    |    |    |    |    |    |    |-- Line

## load and apply SQL Query

In [4]:
with open("spark_extract.sql") as fr:
     query = fr.read()
output = sqlContext.sql(query) 

## Flatten the list

### A

In [ ]:
import itertools
def extract(parts):
    for p in parts:
        for o in itertools.izip(p.Line,p.Latitude,p.Longitude,p.RecordedAtTime,p.vehicleID,p.Trip,p.TripDate):
            yield o

### B

In [31]:
def parse_list(p):
    if p.Line!=None:
        return zip(p.Line,p.Latitude,p.Longitude,p.RecordedAtTime,p.vehicleID,p.Trip,p.TripDate,p.TripPattern,p.MonitoredCallRef,p.DistFromCall,p.CallDistAlongRoute,p.PresentableDistance)
    else:
        return []

## Remove the prefix, timezones and save as CSV

In [50]:
from scipy.interpolate import interp1d
import dateutil.parser

In [ ]:
import time
import dateutil.parser

In [84]:
def unix_time(x):
    dt = dateutil.parser.parse(x)
    return time.mktime(dt.timetuple())

In [117]:
a = [1,2,3,4,5]
b = [2,3,4,5,6]
f = interp1d(a, b)
f(3.5)

array(4.5)

In [102]:
sorted([1,3,4,3,2])

[1, 2, 3, 3, 4]

In [132]:
def predict(x):
    if len(x) >= 2:
        pre_y = [unix_time(p[3]) for p in x ]
        pre_x = [p[-2] for p in x ]
        stops = list(set(p[-2]+p[-3] for p in x))
        f = interp1d(pre_x, pre_y)
    return f(stops) 

In [133]:
output.flatMap(parse_list)\
      .map(lambda x:((x[5],x[6]),x)).groupByKey().mapValues(list)\
      .map(lambda x: predict(x[1])).collect()
      #.map(lambda x: ",".join(map(str, x)))\
      #.map(lambda x: x.replace('MTA NYCT_', '').replace('MTABC_','').replace('MTA_','').replace('-05:00',''))\

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 86.0 failed 1 times, most recent failure: Lost task 2.0 in stage 86.0 (TID 176, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/bonanyuan/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/bonanyuan/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/bonanyuan/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-133-4fd7d40296d7>", line 1, in <lambda>
  File "<ipython-input-132-cb1c8418d13b>", line 7, in predict
  File "/Users/bonanyuan/anaconda/lib/python2.7/site-packages/scipy/interpolate/polyint.py", li

## Import dependencies

In [13]:
from pyspark.sql.types import *

## Reset Schemas and Indexing

In [18]:
customSchema = StructType([StructField("Line", StringType(), True),\
                           StructField("Latitude", DoubleType(), True),\
                           StructField("Longitude", DoubleType(), True),\
                           StructField("RecordedAtTime", StringType(), True),\
                           StructField("vehicleID", StringType(), True),\
                           StructField("TripRef", StringType(), True),\
                           StructField("TripDate", DateType(), True),\
                           StructField("TripPattern", StringType(), True),\
                           StructField("Stop_ID", StringType(), True),\
                           StructField("DistFromCall", DoubleType(), True),\
                           StructField("CallDistAlongRoute", DoubleType(), True),\
                           StructField("PresentableDistance", StringType(), True)])                 

## Use CSV=>DF tool to read saved csv

In [19]:
record = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('final.csv', schema = customSchema)

In [20]:
record.registerTempTable('record')

In [21]:
record.take(1)

[Row(Line=u'B67', Latitude=40.664322, Longitude=-73.983724, RecordedAtTime=u'2015-12-30T18:59:52.000', vehicleID=u'406', TripRef=u'JG_W5-Weekday-109900_B6769_24', TripDate=datetime.date(2015, 12, 30), TripPattern=u'B670106', Stop_ID=u'305679', DistFromCall=12.05, CallDistAlongRoute=8002.2, PresentableDistance=u'at stop')]

In [22]:
import pyspark.sql.functions as f

In [23]:
record_new = record.withColumn("Record_time_unix", f.unix_timestamp('RecordedAtTime', format="yyyy-MM-dd'T'HH:mm:ss.SSS"))
record_new.registerTempTable('record')

In [24]:
record_new.take(1)

[Row(Line=u'B67', Latitude=40.664322, Longitude=-73.983724, RecordedAtTime=u'2015-12-30T18:59:52.000', vehicleID=u'406', TripRef=u'JG_W5-Weekday-109900_B6769_24', TripDate=datetime.date(2015, 12, 30), TripPattern=u'B670106', Stop_ID=u'305679', DistFromCall=12.05, CallDistAlongRoute=8002.2, PresentableDistance=u'at stop', Record_time_unix=1451519992)]

In [32]:
gaps = sqlContext.sql('select TripRef, Record_time_unix, DistFromCall,CallDistAlongRoute,Stop_ID\
                       from record\
                       ORDER BY Record_time_unix DESC')

In [33]:
gaps.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.execution.EvaluatePython.takeAndServe.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange rangepartitioning(Record_time_unix#49L DESC,200), None
+- ConvertToSafe
   +- Project [TripRef#42,unixtimestamp(RecordedAtTime#40,yyyy-MM-dd'T'HH:mm:ss.SSS) AS Record_time_unix#49L,DistFromCall#46,CallDistAlongRoute#47,Stop_ID#45]
      +- Scan CsvRelation(<function0>,Some(final.csv),true,,,",null,#,PERMISSIVE,COMMONS,false,false,false,StructType(StructField(Line,StringType,true), StructField(Latitude,DoubleType,true), StructField(Longitude,DoubleType,true), StructField(RecordedAtTime,StringType,true), StructField(vehicleID,StringType,true), StructField(TripRef,StringType,true), StructField(TripDate,DateType,true), StructField(TripPattern,StringType,true), StructField(Stop_ID,StringType,true), StructField(DistFromCall,DoubleType,true), StructField(CallDistAlongRoute,DoubleType,true), Struct

In [ ]:
gaps.groupby('TripRef')

In [ ]:
gaps.take(1)

In [ ]:
stop_times.printSchema()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
record_new.registerTempTable('record')

In [ ]:
combine = sqlContext.sql(
    'SELECT record.TripRef, record.Stop_ID, record.RecordedAtTime \
    FROM record \
    JOIN stop_times \
    on (record.Stop_ID = stop_times.stop_id AND record.TripRef = stop_times.trip_id)')

In [ ]:
combine.take(1)

In [ ]:
df.select(f.split(df.RecordedAtTime, '-05:00')[0]).alias('RecordedAtTime').take(1)

In [ ]:
timeFmt = 
recordtime = f.unix_timestamp('RecordedAtTime', format=timeFmt)

In [ ]:
record.withColumn("Record_time_unix", f.unix_timestamp('RecordedAtTime', format="yyyy-MM-dd'T'HH:mm:ss.SSS"))

In [ ]:
df.show(2)

In [ ]:
#output_df = output1.toDF(['Line','Lat','Lon','Recordtime','ID','Trip','TripDate'])

In [ ]:
#output_df.show()